You need to download

In [ ]:
!pip3 install --upgrade huggingface-hub transformers==4.56 torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 4.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0 requires torch==2.6.0, but you have torch 2.9.0 which is incompatible.
torchvision 0.21.0 requires torch==2.6.0, but you have torch 2.9.0 which is incompati

then login and get a token from the huggingface user access tokens page:

In [11]:
#uncomment if you haven't already logged in.
#from huggingface_hub import login
#login()
# can also do this from login with hf auth login

In [2]:
import transformers
import torch
import json, sys, os

We w

In [9]:
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


RuntimeError: NVML_SUCCESS == DriverAPI::get()->nvmlInit_v2_() INTERNAL ASSERT FAILED at "/pytorch/c10/cuda/CUDACachingAllocator.cpp":1090, please report a bug to PyTorch. 

In [13]:
messages = [
    {"role": "user", "content": "Who are you?"},
]

In [14]:
messages = pipe(messages)[0]['generated_text']
messages

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'user', 'content': 'Who are you?'},
 {'role': 'assistant',
  'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]

In [15]:
messages.append({'role': 'user', 'content': 'cool!'})
messages = pipe(messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
messages

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'},
   {'role': 'user', 'content': 'cool!'},
   {'role': 'assistant',
    'content': "I'm here to help answer questions, provide information, and even have a conversation. What's on your mind? Want to talk about something specific or just see where the conversation goes?"}]}]

Here is another interesting model:

In [20]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

model_path = "nanonets/Nanonets-OCR2-3B"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto", 
    attn_implementation="sdpa"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

video_preprocessor_config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

In [22]:
def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""
    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

![blah](random_eq.jpeg)

In [40]:
out = ocr_page_with_nanonets_s('/home/louis/research/hf_demo/random_eq.jpeg', model, processor)

In [41]:
out

'<header>Scattering Region WKB</header>\n\nIn the scattering region we have\n\n$$\\begin{bmatrix} \\Delta & P(\\frac{\\partial}{\\partial x}, k_y) \\\\ P^*(\\frac{\\partial}{\\partial x}, k_y) & -\\Delta \\end{bmatrix}\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix} + V(x)\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix} = E\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix}$$\n\nwriting it out\n\n$$\\left(A_{xx}\\frac{\\partial^2}{\\partial x^2} + A_x\\frac{\\partial}{\\partial x} + A_0\\right)v(x) + \\left(\\Delta + V(x)\\right)u(x) = Eu(x)$$\n\n$$\\left(A_{xx}^*\\frac{\\partial^2}{\\partial x^2} + A_x^*\\frac{\\partial}{\\partial x} + A_0^*\\right)u(x) + \\left(-\\Delta + V(x)\\right)v(x) = Ev(x)$$\n\n$$\\frac{\\partial^2v}{\\partial x^2} = -\\frac{A_x}{A_{xx}}\\frac{\\partial v}{\\partial x} - \\frac{A_0}{A_{xx}}v - \\frac{1}{A_{xx}}(\\Delta + V(x) - E)u(x) = A\\frac{\\partial v}{\\partial x} + B(E)u + Fv$$\n\n$$\\frac{\\partial^2u}{\\partial x^2} = -\\frac{A_x^*}{A_{xx}^*}\\frac{\\partial u

\n\nIn the scattering region we have\n\n$$\begin{bmatrix} \Delta & P(\frac{\partial}{\partial x}, k_y) \\ P^*(\frac{\partial}{\partial x}, k_y) & -\Delta \end{bmatrix}\begin{bmatrix} u(x) \\ v(x) \end{bmatrix} + V(x)\begin{bmatrix} u(x) \\ v(x) \end{bmatrix} = E\begin{bmatrix} u(x) \ v(x) \end{bmatrix}$$\n\nwriting it out\n\n$$\left(A_{xx}\frac{\partial^2}{\partial x^2} + A_x\frac{\partial}{\partial x} + A_0\right)v(x) + \left(\Delta + V(x)\right)u(x) = Eu(x)$$\n\n$$\left(A_{xx}^*\frac{\partial^2}{\partial x^2} + A_x^*\frac{\partial}{\partial x} + A_0^*\right)u(x) + \left(-\Delta + V(x)\right)v(x) = Ev(x)$$\n\n$$\frac{\partial^2v}{\partial x^2} = -\frac{A_x}{A_{xx}}\frac{\partial v}{\partial x} - \frac{A_0}{A_{xx}}v - \frac{1}{A_{xx}}(\Delta + V(x) - E)u(x) = A\frac{\partial v}{\partial x} + B(E)u + Fv$$\n\n$$\frac{\partial^2u}{\partial x^2} = -\frac{A_x^*}{A_{xx}^*}\frac{\partial u}{\partial x} - \frac{A_0^*}{A_{xx}^*}u - \frac{1}{A_{xx}^*}(-\Delta + V(x) - E)v(x) = A^*\frac{\partial u}{\partial x} + C(E)v + F^*u$$